In [16]:
# <editor-fold desc="import stuff">
import pandas as pd
import numpy as np
import io
# </editor-fold>

In [17]:
# <editor-fold desc="Make Dataframe">
path = "../untitled3/DatafinitiElectronicsProductsPricingData.csv"
Review_df = pd.read_csv(path)
# </editor-fold>


In [18]:
Review_df.columns

Index(['id', 'prices.amountMax', 'prices.amountMin', 'prices.availability',
       'prices.condition', 'prices.dateSeen', 'prices.isSale',
       'prices.merchant', 'prices.shipping', 'asins', 'brand', 'categories',
       'dateAdded', 'dateUpdated', 'ean', 'keys', 'manufacturer',
       'manufacturerNumber', 'upc', 'weight'],
      dtype='object')

In [19]:
# <editor-fold desc="train, test validate part here">
# <editor-fold desc="train_test_split model">
from sklearn.model_selection import train_test_split
#let train, validate and test one dataframe into .70, .20, .10
train_ratio = .70
validation_ratio = .20
test_ratio = .10

train, test = train_test_split(Review_df, test_size = 1 - train_ratio)
validate, test = train_test_split(test, test_size=test_ratio/(test_ratio+validation_ratio))
# </editor-fold>
# </editor-fold>

In [20]:
train.dtypes

id                      object
prices.amountMax       float64
prices.amountMin       float64
prices.availability     object
prices.condition        object
prices.dateSeen         object
prices.isSale             bool
prices.merchant         object
prices.shipping         object
asins                   object
brand                   object
categories              object
dateAdded               object
dateUpdated             object
ean                     object
keys                    object
manufacturer            object
manufacturerNumber      object
upc                     object
weight                 float64
dtype: object

#let encode a single string column.

In [21]:
# <editor-fold desc="looks at some columns">
id = train['id'].value_counts()
prices_availability = train['prices.availability'].value_counts()
prices_condition =train['prices.condition'].value_counts()
prices_merchant =train['prices.merchant'].value_counts()
prices_shipping =train['prices.shipping'].value_counts()
categories= train['categories'].value_counts()
manufacturer =train['manufacturer'].value_counts()
manufacturerNumber =train['manufacturerNumber'].value_counts()
# </editor-fold>


In [22]:
# <editor-fold desc="look at id">
#print(prices_availability, prices_condition, prices_merchant, prices_shipping, categories, manufacturer, manufacturerNumber)
# </editor-fold>


OneHotEncode = prices.availability, prices.condition, 


Rewrite columns for manufacturer, 
prices.shipping, if has free shipping = 1, if no free shipping 0.
prices.merchant = 
manufacturer,

drop categories,

In [23]:
# <editor-fold desc="Cardinality and target">
target = 'prices.amountMax'

train_features = train.drop(columns=[target,'id', 'prices.amountMin'])
#In the dataframe, grab all the numeric columns, then list it into columns. Finally turn it into array.
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
#This (not a number) columns has how many unique categories?
cardinality = train_features.select_dtypes(exclude='number').nunique()
#if this columns has less than 50 keep that column, and turn it into array. (important step for panda ARRAY)
categorical_features = cardinality[cardinality <= 50].index.tolist()
#combine the list
features = numeric_features + categorical_features
# </editor-fold>

In [24]:
# <editor-fold desc="X train, validate, and test">
X_train = train[features]
y_train = train[target]

X_validate = validate[features]
y_validate = validate[target]

X_test = test[features]
# </editor-fold>

instantiate it and ensure that we get a dense (and not sparse) array returned, and then encode our single column with the fit_transform method.

In [25]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

pipeline = make_pipeline(
    preprocessing.LabelEncoder(),
    #ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1)
)

In [26]:
# <editor-fold desc="This is to fix continous Error">
#from sklearn import preprocessing
#from sklearn import utils

#lab_enc = preprocessing.LabelEncoder()
#training_scores_encoded = lab_enc.fit_transform(y_train)
#print(training_scores_encoded)
#print(utils.multiclass.type_of_target(y_train))
#print(utils.multiclass.type_of_target(y_train.astype('int')))
#print(utils.multiclass.type_of_target(training_scores_encoded))

# </editor-fold>


import xgboost

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore') #handle_unknow by putting all zeors.
for one unique in a column, nan, none, 

can fill value with a paramenter
si = SimpleImputer(strategy='constant', fill_value = 'MISSING')

The OneHotEncoder works analogously. During the fit method, it finds all the unique values for each column and again stores this. When transform is called, it uses these stored unique values to produce the binary array.

In [44]:
from xgboost import XGBRegressor

pipeline_OHE_XGRegressor = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    XGBRegressor()
)

In [46]:
pipeline_OHE_XGRegressor.fit(X_train,y_train)

C:\Users\D3MoNa\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\D3MoNa\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[02:16:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=['prices.availability', 'prices.condition', 'prices.shipping'],
       drop_invariant=False, handle_missing='value',
       handle_unknown='value', return_df=True, use_cat_names=True,
       verbose=0)), ('xgbregressor', XGBRegressor(base_score=0.5, booste...lpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1))])

In [60]:
from sklearn.metrics import r2_score
#from xgboost import r2_score
y_pred_OHE_XGregressor = pipeline_OHE_XGRegressor.predict(X_validate)
print('validation Accuracy', r2_score(y_validate,y_pred_OHE_XGregressor))

validation Accuracy 0.6574682169766535


In [74]:
encoder = ce.OneHotEncoder()
X_train_encoder = encoder.fit_transform(X_train)
X_val_encoder = encoder.transform(X_validate)

In [78]:
eval_set = [(X_train_encoder, y_train),
        (X_val_encoder, y_validate)]

model = XGBRegressor(
   
)

model.fit(X_train_encoder, y_train, eval_set=eval_set)

[03:23:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:883.543	validation_1-rmse:931.438
[1]	validation_0-rmse:831.467	validation_1-rmse:879.296
[2]	validation_0-rmse:786.326	validation_1-rmse:832.752
[3]	validation_0-rmse:747.616	validation_1-rmse:793.202
[4]	validation_0-rmse:714.432	validation_1-rmse:760.292
[5]	validation_0-rmse:686.17	validation_1-rmse:731.667
[6]	validation_0-rmse:660.317	validation_1-rmse:704.152
[7]	validation_0-rmse:637.361	validation_1-rmse:683.101
[8]	validation_0-rmse:618.917	validation_1-rmse:664.921
[9]	validation_0-rmse:603.58	validation_1-rmse:650.196
[10]	validation_0-rmse:590.077	validation_1-rmse:636.76
[11]	validation_0-rmse:578.076	validation_1-rmse:626.182
[12]	validation_0-rmse:569.184	validation_1-rmse:618.195
[13]	validation_0-rmse:560.089	validation_1-rmse:610.004
[14]	validation_0-rmse:553.342	validation_1-rmse:603.828
[15]	validation_0-rmse:544.198	validation_

C:\Users\D3MoNa\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\D3MoNa\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[59]	validation_0-rmse:460.509	validation_1-rmse:513.325
[60]	validation_0-rmse:459.399	validation_1-rmse:512.768
[61]	validation_0-rmse:458.439	validation_1-rmse:511.664
[62]	validation_0-rmse:458.281	validation_1-rmse:511.569
[63]	validation_0-rmse:458.087	validation_1-rmse:511.396
[64]	validation_0-rmse:457.142	validation_1-rmse:511.009
[65]	validation_0-rmse:456.218	validation_1-rmse:510.651
[66]	validation_0-rmse:456.054	validation_1-rmse:510.501
[67]	validation_0-rmse:455.268	validation_1-rmse:509.257
[68]	validation_0-rmse:454.496	validation_1-rmse:508.859
[69]	validation_0-rmse:454.357	validation_1-rmse:508.74
[70]	validation_0-rmse:453.972	validation_1-rmse:508.293
[71]	validation_0-rmse:453.707	validation_1-rmse:508.234
[72]	validation_0-rmse:453.138	validation_1-rmse:507.777
[73]	validation_0-rmse:452.121	validation_1-rmse:506.659
[74]	validation_0-rmse:450.233	validation_1-rmse:504.107
[75]	validation_0-rmse:450.117	validation_1-rmse:504.085
[76]	validation_0-rmse:449.869	v

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)